In [7]:
# Use sklearn version 1.3.2 and scikeras 0.12.0 to make sure the two are compatible, and will also work with tensorflow.
!pip install scikit-learn scikeras

from scikeras.wrappers import KerasRegressor

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# So, we are going back to the Boston Housing data here.
from tensorflow.keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = (boston_housing.load_data())

#*Grid Search CV With Keras Model*

In [8]:
# Make sure you set your custom parameters for training as arguments in your model creation function.
def create_model(loss="mse",optimizer="sgd",output_activation="relu",units=100,numLayers=2, batch_size=10):

    # I beleve that you need to explicitly declare an input layer for the scikeras wrapper to work...
    model = keras.Sequential([
        layers.Input(shape=(train_data.shape[1],)),
        layers.Dense(units, activation="relu")
    ])

    if numLayers == 2:
        model.add(layers.Dense(units, activation="relu"))

    model.add(layers.Dense(1, activation=output_activation))

    model.compile(loss="mse",optimizer=optimizer, metrics=['mae'])
    return model

# You also need to specify the 'custom' parameters here that you want to add, for them to show up as a trainable parameter in GridSearchCV.
regf = KerasRegressor(model=create_model, loss="mse", optimizer="sgd", output_activation="relu", units=100, numLayers=2, batch_size=10, verbose=0)

# Here are the configurable parameters we can now search over for either object.
print(regf.get_params().keys())

dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs', 'output_activation', 'units', 'numLayers'])


And this is how I would invoke my grid search...

In [9]:
from sklearn.model_selection import GridSearchCV

# Because we are creating the models but are not compiling them yet (we will let the grid fit compile the models on the fly),
# this will produce a bunch of warnings. I'm just suppressing the warnings.
#import logging, os
#logging.disable(logging.WARNING)
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

params = {
    "numLayers": [1,2],
    "units": [100,125]
    #"activation": ['relu','selu',None],
    #"batch_size": [25,50],
    #"epochs": [10,20,30]
}

grid = GridSearchCV(regf, params, scoring='neg_mean_absolute_error',verbose=11,cv=3,error_score='raise')

grid.fit(train_data, train_targets)

print(grid.best_score_, grid.best_params_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV 1/3; 1/4] START numLayers=1, units=100......................................


ValueError: Could not interpret metric identifier: loss

I can then extract the parameters that yielded the top performance...

In [ ]:
print(f"Best Score  : {grid.best_score_}")
print(f"Best Params : {grid.best_params_}")

Best Score  : -22.398225308641976
Best Params : {'numLayers': 1, 'units': 100}


Finally, a little function that looks at pairs of parameter values, and the associated model performance, holding all other parameters to their ideal values.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def plot_results(index='units', columns='activation'):
    index = 'param_' + index
    columns = 'param_' + columns

    # prepare the results into a pandas.DataFrame
    df = pd.DataFrame(grid.cv_results_)

    # Remove the other by selecting their best values (from gscv.best_params_)
    other = [c for c in df.columns if c[:6] == 'param_']
    other.remove(index)
    other.remove(columns)

    # Set all other parameters to their "top" values.
    for col in other:
        df = df[df[col] == grid.best_params_[col[6:]]]

    # Create pivot tables for easy plotting
    table_mean = df.pivot_table(index=index, columns=columns,
                                values=['mean_test_score'])

    # plot the pivot tables
    plt.figure()
    ax = plt.gca()
    for col_mean in table_mean.columns:
        table_mean[col_mean].plot(marker='o',label=col_mean)
    plt.title('Grid-search results (higher is better)')
    plt.ylabel('- Mean Absolute Error')
    plt.legend(title=table_mean.columns.names)
    plt.show()

plot_results(index='units', columns='numLayers')